In [1]:
!pip install --upgrade xgboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 6.4 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.2 MB/s eta 0:00:00


In [2]:
from tensorflow.keras import regularizers

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

import numpy as np
import time
import os
import copy
import pandas
import pandas as pd

import xgboost as xgb
import catboost as cab
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
tf.__version__, np.__version__, pd.__version__, xgb.__version__, cab.__version__, sklearn.__version__

('2.9.2', '1.21.6', '1.3.5', '1.7.3', '1.1.1', '1.0.2')

In [ ]:
!python --version

Python 3.8.16


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd drive/MyDrive/tabular_playground

Mounted at /content/drive
/content/drive/MyDrive/tabular_playground


In [5]:
csv = pd.read_csv('train.csv') # get dataset
csv

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26565,26565,E,158.95,material_7,material_6,6,9,6,16,4,...,NaN,12.177,17.942,10.112,15.795,18.572,16.144,NaN,729.131,0
26566,26566,E,146.02,material_7,material_6,6,9,10,12,8,...,11.242,14.179,20.564,10.234,14.450,14.322,13.146,16.471,853.924,0
26567,26567,E,115.62,material_7,material_6,6,9,1,10,1,...,11.407,16.437,17.476,8.668,15.069,16.599,15.590,14.065,750.364,0
26568,26568,E,106.38,material_7,material_6,6,9,2,9,4,...,11.392,17.064,17.814,14.928,16.273,15.485,13.624,12.865,730.156,0


In [6]:
columns = csv.columns

In [7]:
attr_map = {'material_5' : 0, 'material_6' : 1, 'material_7' : 2, 'material_8' : 3,} # to map string to categorical number

In [8]:
csv['attribute_0'] = csv['attribute_0'].map(attr_map)
csv['attribute_1'] = csv['attribute_1'].map(attr_map)

In [9]:
for column in columns:
  if csv[column].isnull().any():
    # print(csv[column].mean)
    temp = csv[column].to_numpy()
    temp = temp[~np.isnan(temp)]
    csv[column] = csv[column].fillna(np.mean(temp)) # fill nan
    print(np.mean(temp))
    print(csv[column].isnull().any())

127.82623252279637
False
17.79152751155065
False
11.731987861094039
False
17.1278036224608
False
17.510758787925816
False
11.71662411734873
False
19.024713893895463
False
11.430724539320522
False
16.117710565888405
False
19.17208545135846
False
11.702464455925348
False
15.652903573156959
False
16.048444120505344
False
14.995554293391963
False
16.460726941946035
False
701.2690585522523
False


In [10]:
X = csv.to_numpy()[:, 2:25] # using only the 3 - 25 columns
Y = csv.to_numpy()[:, 25]
X.shape, Y.shape

((26570, 23), (26570,))

In [11]:
# std = []
# mean = []
# for attr in range(5, X.shape[1]):
#   std.append(np.std(X[:, attr]))
#   mean.append(np.mean(X[:, attr]))
#   X[:, attr] = (X[:, attr] - mean[-1]) / std[-1]
  # print(std[-1], mean[-1])

In [12]:
# x_pass = X[Y == 1]
# y_pass = Y[Y == 1]
# x_failure = X[Y == 0]
# y_failure = Y[Y == 0]
# x_pass.shape, x_failure.shape

In [14]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.05, shuffle = True)

In [15]:
x_pass = x_train[y_train == 1]
y_pass = y_train[y_train == 1]
x_failure = x_train[y_train == 0]
y_failure = y_train[y_train == 0]
x_pass.shape, x_failure.shape, y_failure.shape, y_pass.shape

((5361, 23), (19880, 23), (19880,), (5361,))

In [16]:
# over sampling
x_amp = x_failure.copy() # [:7000]
y_amp = y_failure.copy() # [:7000]
for i in range(10):
    x_amp = np.concatenate([x_amp, x_pass], axis = 0)
    y_amp = np.concatenate([y_amp, y_pass], axis = 0)
x_amp.shape, y_amp.shape

((73490, 23), (73490,))

In [ ]:
# hyperparameters, not recommended to change
# n_estimators = 104590
# max_depth = 1
# gamma = 0.42
# colsample_bytree = 0.6
# alpha = 1
# booster = 'gbtree'
# learning_rate = 1

In [ ]:
# model = XGBClassifier(n_estimators = n_estimators, max_depth = max_depth, learning_rate = learning_rate, booster = booster, gamma = gamma, colsample_bytree = colsample_bytree)

In [ ]:
# model_cat = CatBoostClassifier(learning_rate = 0.01, max_depth = 9)

In [ ]:
# model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.42, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=1, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=104590, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
# accuracy_score(y_test, model.predict(x_test)), accuracy_score(y_failure, model.predict(x_failure)) # check xgboost performance

(0.6877351392024078, 0.6595958580476525)

In [ ]:
# model_cat.fit(x_train, y_train) #
# accuracy_score(y_test, model_cat.predict(x_test)), accuracy_score(y_failure, model_cat.predict(x_failure)) # check catboost performance

In [ ]:
# oversampling test
# for repeat in range(5): 
#   x_amp = x_failure.copy()
#   y_amp = y_failure.copy()
#   for i in range(repeat):
#     x_amp = np.concatenate([x_amp, x_pass], axis = 0)
#     y_amp = np.concatenate([y_amp, y_pass], axis = 0)
#   model = XGBClassifier(n_estimators = n_estimators, max_depth = max_depth, learning_rate = learning_rate, booster = booster, gamma = gamma, colsample_bytree = colsample_bytree) 
#   model.fit(x_amp, y_amp)
#   print(accuracy_score(y_pass, model.predict(x_pass)), accuracy_score(y_failure, model.predict(x_failure)))

0.0 1.0
0.003750234389649353 0.9994474583082178
0.08644290268141759 0.9718706047819972
0.3678979936246015 0.7971167369901547
0.6561035064691543 0.5369198312236287


In [ ]:
# for n in range(100): # testing gamma
#   model = XGBClassifier(n_estimators = n_estimators, max_depth = max_depth, learning_rate = 1, booster = booster, gamma = n / 100, colsample_bytree = 0.6) 
#   model.fit(x_amp, y_amp)
#   print(accuracy_score(y_pass, model.predict(x_pass)), accuracy_score(y_failure, model.predict(x_failure)))

0.7069384701701888 0.5909319392781743
0.7069384701701888 0.5909319392781743
0.7069384701701888 0.5909319392781743
0.7069384701701888 0.5909319392781743


KeyboardInterrupt: ignored

In [ ]:
net = models.Sequential([ # neural network

    layers.Dense(23, input_shape = (23,)),
    # layers.BatchNormalization(),
    layers.Dense(64, kernel_regularizer = regularizers.L1(1e-5)),
    # layers.LeakyReLU(alpha = 0.01),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(128, kernel_regularizer = regularizers.L1(1e-5)),
    layers.LeakyReLU(alpha = 0.01),
    layers.Dense(256, kernel_regularizer = regularizers.L1(1e-5)),
    layers.LeakyReLU(alpha = 0.01),
    layers.Dropout(0.25),
    # layers.Dense(256, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5)),
    # layers.LeakyReLU(alpha = 0.01),
    # layers.BatchNormalization(),
    # layers.Dense(128, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5)),
    # layers.LeakyReLU(alpha = 0.01),
    # layers.Dropout(0.2),
    # layers.Dense(256, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5)),
    # layers.LeakyReLU(alpha = 0.01),
    # layers.BatchNormalization(),
    # layers.Dense(512, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5)),
    # layers.LeakyReLU(alpha = 0.01),
    # layers.Dropout(0.2),
    layers.Dense(256, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5), activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dense(128, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5), activation = 'relu'),
    layers.Dense(256, kernel_regularizer = regularizers.L1(1e-5), activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dense(128, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5), activation = 'relu'),
    layers.Dropout(0.25),
    layers.Dense(64, kernel_regularizer = regularizers.L1(1e-5), activation = 'relu'),
    layers.Dense(32, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5), activation = 'relu'),
    layers.Dense(1, activation = 'sigmoid'),
    # layers.Dense(1, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5)),
])

In [ ]:
net.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_141 (Dense)           (None, 23)                552       
                                                                 
 dense_142 (Dense)           (None, 64)                1536      
                                                                 
 batch_normalization_46 (Bat  (None, 64)               256       
 chNormalization)                                                
                                                                 
 dense_143 (Dense)           (None, 128)               8320      
                                                                 
 leaky_re_lu_23 (LeakyReLU)  (None, 128)               0         
                                                                 
 dense_144 (Dense)           (None, 256)               33024     
                                                      

In [17]:
input = layers.Input(shape=(23,)) # residual neural network
res = layers.Dense(64, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(input)
x1 = layers.Dense(64, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(input)
x1 = layers.LeakyReLU(alpha = 0.01)(x1)
x1 = layers.Dropout(0.25)(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.Dense(32, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x1)
x1 = layers.Dense(64, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x1)
x1 = layers.LeakyReLU(alpha = 0.01)(x1)
x1 = layers.Dropout(0.2)(x1)
x1 = layers.Dense(128, activation = 'relu', kernel_regularizer = regularizers.L1(1e-5))(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.Dense(128, activation = 'relu', kernel_regularizer = regularizers.L1(1e-5))(x1)
x1 = layers.Dropout(0.2)(x1)
x1 = layers.Dense(32, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x1)
x1 = layers.Dense(64, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x1)
x1 = layers.BatchNormalization()(x1)

x2 = layers.Dense(64, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(input)
x2 = layers.LeakyReLU(alpha = 0.01)(x2)
x2 = layers.Dropout(0.25)(x2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.Dense(32, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x2)
x2 = layers.Dense(64, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x2)
x2 = layers.LeakyReLU(alpha = 0.01)(x2)
x2 = layers.Dropout(0.2)(x2)
x2 = layers.Dense(128, activation = 'relu', kernel_regularizer = regularizers.L1(1e-5))(x2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.Dense(128, activation = 'relu', kernel_regularizer = regularizers.L1(1e-5))(x2)
x2 = layers.Dropout(0.2)(x2)
x2 = layers.Dense(32, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x2)
x2 = layers.Dense(64, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x2)
x2 = layers.BatchNormalization()(x2)

x3 = layers.Dense(64, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(input)
x3 = layers.LeakyReLU(alpha = 0.01)(x3)
x3 = layers.Dropout(0.25)(x3)
x3 = layers.BatchNormalization()(x3)
x3 = layers.Dense(32, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x3)
x3 = layers.Dense(64, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x3)
x3 = layers.LeakyReLU(alpha = 0.01)(x3)
x3 = layers.Dropout(0.2)(x3)
x3 = layers.Dense(128, activation = 'relu', kernel_regularizer = regularizers.L1(1e-5))(x3)
x3 = layers.BatchNormalization()(x3)
x3 = layers.Dense(128, activation = 'relu', kernel_regularizer = regularizers.L1(1e-5))(x3)
x3 = layers.Dropout(0.2)(x3)
x3 = layers.Dense(32, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x3)
x3 = layers.Dense(64, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x3)
x3 = layers.BatchNormalization()(x3)

x4 = layers.Dense(64, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(input)
x4 = layers.LeakyReLU(alpha = 0.01)(x4)
x4 = layers.Dropout(0.25)(x4)
x4 = layers.BatchNormalization()(x4)
x4 = layers.Dense(32, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x4)
x4 = layers.Dense(64, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x4)
x4 = layers.LeakyReLU(alpha = 0.01)(x4)
x4 = layers.Dropout(0.2)(x4)
x4 = layers.Dense(128, activation = 'relu', kernel_regularizer = regularizers.L1(1e-5))(x4)
x4 = layers.BatchNormalization()(x4)
x4 = layers.Dense(128, activation = 'relu', kernel_regularizer = regularizers.L1(1e-5))(x4)
x4 = layers.Dropout(0.2)(x4)
x4 = layers.Dense(32, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x4)
x4 = layers.Dense(64, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x4)
x4 = layers.BatchNormalization()(x4)

x5 = layers.Dense(64, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(input)
x5 = layers.LeakyReLU(alpha = 0.01)(x5)
x5 = layers.Dropout(0.25)(x5)
x5 = layers.BatchNormalization()(x5)
x5 = layers.Dense(32, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x5)
x5 = layers.Dense(64, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x5)
x5 = layers.LeakyReLU(alpha = 0.01)(x5)
x5 = layers.Dropout(0.2)(x5)
x5 = layers.Dense(128, activation = 'relu', kernel_regularizer = regularizers.L1(1e-5))(x5)
x5 = layers.BatchNormalization()(x5)
x5 = layers.Dense(128, activation = 'relu', kernel_regularizer = regularizers.L1(1e-5))(x5)
x5 = layers.Dropout(0.2)(x5)
x5 = layers.Dense(32, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x5)
x5 = layers.Dense(64, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x5)
x5 = layers.BatchNormalization()(x5)

x6 = layers.Dense(64, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(input)
x6 = layers.LeakyReLU(alpha = 0.01)(x6)
x6 = layers.Dropout(0.25)(x6)
x6 = layers.BatchNormalization()(x6)
x6 = layers.Dense(32, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x6)
x6 = layers.Dense(64, kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x6)
x6 = layers.LeakyReLU(alpha = 0.01)(x6)
x6 = layers.Dropout(0.2)(x6)
x6 = layers.Dense(128, activation = 'relu', kernel_regularizer = regularizers.L1(1e-5))(x6)
x6 = layers.BatchNormalization()(x6)
x6 = layers.Dense(128, activation = 'relu', kernel_regularizer = regularizers.L1(1e-5))(x6)
x6 = layers.Dropout(0.2)(x6)
x6 = layers.Dense(32, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x6)
x6 = layers.Dense(64, activation = 'relu', kernel_regularizer = regularizers.L1L2(1e-5, 1e-5))(x6)
x6 = layers.BatchNormalization()(x6)

# equivalent to `added = tf.keras.layers.add([x1, x2])`
added = tf.keras.layers.Concatenate()([x1, x2, x3, x4, x5, x6])
out = layers.Dense(64, activation = 'relu', kernel_regularizer = regularizers.L1(1e-5))(added)
out = layers.BatchNormalization()(out)
out = layers.Dense(32, activation = 'relu', kernel_regularizer = regularizers.L1(1e-5))(out)
out = layers.Dense(1, activation = 'sigmoid')(out)
residual = tf.keras.models.Model(inputs = input, outputs = out)

In [18]:
residual.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 23)]         0           []                               
                                                                                                  
 dense_47 (Dense)               (None, 64)           1536        ['input_2[0][0]']                
                                                                                                  
 dense_54 (Dense)               (None, 64)           1536        ['input_2[0][0]']                
                                                                                                  
 dense_61 (Dense)               (None, 64)           1536        ['input_2[0][0]']                
                                                                                            

In [19]:
epochs = 32
batch_size = 32
lr = 4e-4
ema_momentum = 0.9999
use_ema = True
weight_decay = 0.05

In [ ]:
# net.compile(optimizer = keras.optimizers.experimental.AdamW(learning_rate = lr, weight_decay = weight_decay, ema_momentum = ema_momentum, use_ema = use_ema), loss = keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])
net.compile(optimizer = keras.optimizers.Adam(learning_rate = lr), loss = keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [20]:
residual.compile(optimizer = keras.optimizers.Adam(lr), loss = keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [23]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.05, shuffle = True)
x_train = np.asarray(x_train).astype('float32')
x_test = np.asarray(x_test).astype('float32')
y_train = np.asarray(y_train).astype('int32')
y_test = np.asarray(y_test).astype('int32')
x_amp = np.asarray(x_amp).astype('float32')
y_amp = np.asarray(y_amp).astype('int32')
x_pass = np.asarray(x_pass).astype('float32')
y_pass = np.asarray(y_pass).astype('int32')
x_failure = np.asarray(x_failure).astype('float32')
y_failure = np.asarray(y_failure).astype('int32')
X = np.asarray(X).astype('float32')
Y = np.asarray(Y).astype('int32')

In [24]:
class_weight = {0 : (1 / len(x_failure)) * (len(x_train) / 2), 1 : (2.5 / len(x_pass)) * (len(x_train) / 2)}

In [ ]:
net.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = [x_test, y_test], class_weight = class_weight)

Epoch 1/32
789/789 [==============================] - 7s 7ms/step - loss: 1.2083 - accuracy: 0.2203 - val_loss: 1.0001 - val_accuracy: 0.2190
Epoch 2/32
789/789 [==============================] - 7s 9ms/step - loss: 1.1887 - accuracy: 0.2154 - val_loss: 1.1525 - val_accuracy: 0.2190
Epoch 3/32
789/789 [==============================] - 6s 7ms/step - loss: 1.1828 - accuracy: 0.2127 - val_loss: 1.0677 - val_accuracy: 0.2190
Epoch 4/32
789/789 [==============================] - 5s 7ms/step - loss: 1.1826 - accuracy: 0.2137 - val_loss: 1.0573 - val_accuracy: 0.2190
Epoch 5/32
789/789 [==============================] - 5s 7ms/step - loss: 1.1781 - accuracy: 0.2124 - val_loss: 1.0572 - val_accuracy: 0.2190
Epoch 6/32
789/789 [==============================] - 5s 7ms/step - loss: 1.1743 - accuracy: 0.2127 - val_loss: 1.1711 - val_accuracy: 0.2190
Epoch 7/32
789/789 [==============================] - 5s 7ms/step - loss: 1.1714 - accuracy: 0.2124 - val_loss: 1.1326 - val_accuracy: 0.2190
Epoch 

In [25]:
residual.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = [x_test, y_test], class_weight = class_weight)

Epoch 1/32
789/789 [==============================] - 26s 21ms/step - loss: 1.3580 - accuracy: 0.2727 - val_loss: 1.4148 - val_accuracy: 0.2017
Epoch 2/32
789/789 [==============================] - 16s 20ms/step - loss: 1.2955 - accuracy: 0.2234 - val_loss: 1.2316 - val_accuracy: 0.2017
Epoch 3/32
789/789 [==============================] - 19s 24ms/step - loss: 1.2857 - accuracy: 0.2223 - val_loss: 1.2126 - val_accuracy: 0.2017
Epoch 4/32
789/789 [==============================] - 15s 19ms/step - loss: 1.2783 - accuracy: 0.2185 - val_loss: 1.1664 - val_accuracy: 0.2017
Epoch 5/32
789/789 [==============================] - 15s 19ms/step - loss: 1.2746 - accuracy: 0.2155 - val_loss: 1.3375 - val_accuracy: 0.2017
Epoch 6/32
789/789 [==============================] - 15s 19ms/step - loss: 1.2694 - accuracy: 0.2154 - val_loss: 1.2119 - val_accuracy: 0.2017
Epoch 7/32
789/789 [==============================] - 15s 19ms/step - loss: 1.2669 - accuracy: 0.2145 - val_loss: 1.2214 - val_accuracy:

In [ ]:
# net.save('net')

In [26]:
# residual.save('residual')